This notebook runs SExtractor on the test dataset.

In [1]:
import numpy as np
import sep
import tensorflow as tf
import matplotlib.pyplot as plt
from matplotlib.patches import Ellipse

print("Imported")

Imported


In [2]:
image_size = (128, 128)
batch_size = 32
test_ds = tf.keras.preprocessing.image_dataset_from_directory(
    "tfData-testing",
    seed=1337,
    image_size=image_size,
    batch_size=batch_size,
    shuffle = True,
    color_mode="grayscale"
)

Found 4401 files belonging to 2 classes.


In [3]:
# need to reshape test dataset to work with individual images

rows = []
for row in test_ds.unbatch().as_numpy_iterator():
    rows.append(row)

In [6]:
%%time
predictions = []
truth = []

# for each image in the dataset, analyse with sextractor
for (arr, label) in rows:
    #fig, ax = plt.subplots(1,1)
    img = arr[:, :, 0].copy() #remove colour channel, and make copy (SExtractor requires a copy)
    
    #plt.imshow(img)
    
    bkg = sep.Background(img) #calculate background of image
    img_sub = img - bkg # create new image with background subtracted

    objects = sep.extract(img_sub, 1.5, err=bkg.globalrms) # extract all of the objects from the image
    
    if len(objects) > 1: 
        predictions.append("Blended")
    else:
        predictions.append("Unblended")
        
    if label == 0:
        truth.append("Blended")
    else:
        truth.append("Unblended")
    
    
    #for i in range(len(objects)):
    #    e = Ellipse(xy=(objects['x'][i], objects['y'][i]),
    #    width=6*objects['a'][i],
    #    height=6*objects['b'][i],
    #    angle=objects['theta'][i] * 180. / np.pi)
    #    e.set_facecolor('none')
    #    e.set_edgecolor('red')
    #    ax.add_artist(e)

CPU times: total: 16.6 s
Wall time: 16.7 s


In [9]:
%%time

tp = 0
fp = 0
tn = 0
fn = 0

# calculate confusion matrix elements
for i in range(len(predictions)):
    pred = predictions[i]
    tru = truth[i]
    if pred == "Blended" and tru == "Blended":
        tp+=1
    elif pred == "Blended" and tru == "Unblended":
        fp+=1
    elif pred == "Unblended" and tru == "Unblended":
        tn+=1
    elif pred == "Unblended" and tru == "Blended":
        fn+=1

print("Recall ", tp/(tp+fn))
print("Precision ", tp/(tp+fp))

Recall  0.8915
Precision  0.9171810699588477
CPU times: total: 0 ns
Wall time: 2 ms
